# Music Source Separation by Pretrained Models
If you want to separate your own music files, see
- `egs/tutorials/conv-tasnet/separate_music.ipynb` for Conv-TasNet
- `egs/tutorials/mm-dense-lstm/separate_music.ipynb` for MMDenseLSTM
- `egs/tutorials/umx/separate_music.ipynb` for OpenUnmix
- `egs/tutorials/d3net/separate_music.ipynb` for D3Net

In [ ]:
%%shell
git clone https://github.com/tky823/DNN-based_source_separation.git

cd "./DNN-based_source_separation/egs/tutorials/"

# To install torch & torchaudio
pip install -r requirements.txt

In [ ]:
%%shell
# Download music datset
wget "https://zenodo.org/api/files/1ff52183-071a-4a59-923f-7a31c4762d43/MUSDB18-7-STEMS.zip"
unzip "./MUSDB18-7-STEMS.zip"

# Convert .mp4 to .wav
cd "./train"

for stem in *.stem.mp4 ; do
    name=`echo $stem | awk -F".stem.mp4" '{$0=$1}1'`;
    echo "$stem"
    mkdir "$name"
    cd "$name"
    ffmpeg -loglevel panic -i "../${stem}" -map 0:0 -vn mixture.wav
    ffmpeg -loglevel panic -i "../${stem}" -map 0:1 -vn drums.wav
    ffmpeg -loglevel panic -i "../${stem}" -map 0:2 -vn bass.wav
    ffmpeg -loglevel panic -i "../${stem}" -map 0:3 -vn other.wav
    ffmpeg -loglevel panic -i "../${stem}" -map 0:4 -vn vocals.wav
    cd "../"
done

In [ ]:
import sys
sys.path.append("/content/DNN-based_source_separation/src")

In [ ]:
import IPython.display as ipd
import torch
import torchaudio

In [ ]:
from models.conv_tasnet import ConvTasNet
from models.mm_dense_lstm import MMDenseLSTM, ParallelMMDenseLSTM
from models.umx import OpenUnmix, ParallelOpenUnmix
from models.xumx import CrossNetOpenUnmix
from models.d3net import D3Net, ParallelD3Net

In [ ]:
n_sources = 4
name = "ANiMAL - Rockshow"

In [ ]:
waveform_bass, sample_rate = torchaudio.load("/content/train/{}/bass.wav".format(name))
waveform_drums, sample_rate = torchaudio.load("/content/train/{}/drums.wav".format(name))
waveform_other, sample_rate = torchaudio.load("/content/train/{}/other.wav".format(name))
waveform_vocals, sample_rate = torchaudio.load("/content/train/{}/vocals.wav".format(name))

display(ipd.Audio(waveform_bass, rate=sample_rate))
display(ipd.Audio(waveform_drums, rate=sample_rate))
display(ipd.Audio(waveform_other, rate=sample_rate))
display(ipd.Audio(waveform_vocals, rate=sample_rate))

In [ ]:
mixture = waveform_bass + waveform_drums + waveform_other + waveform_vocals
display(ipd.Audio(mixture, rate=sample_rate))

In [ ]:
model = ConvTasNet.build_from_pretrained(task="musdb18", sample_rate=sample_rate)

input = mixture.unsqueeze(dim=0).unsqueeze(dim=1)

model.eval()
with torch.no_grad():
    output = model(input)

output = output.squeeze(dim=0)
estimated = torch.unbind(output, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=sample_rate))

In [ ]:
modules = {}
modules['bass'] = MMDenseLSTM.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='bass')
modules['drums'] = MMDenseLSTM.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='drums')
modules['other'] = MMDenseLSTM.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='other')
modules['vocals'] = MMDenseLSTM.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='vocals')
model = ParallelMMDenseLSTM(modules)
wrapper_model = ParallelMMDenseLSTM.TimeDomainWrapper(model, fft_size=4096, hop_size=1024, window_fn='hann')

input = mixture.unsqueeze(dim=0).unsqueeze(dim=1)

wrapper_model.eval()
with torch.no_grad():
    output = wrapper_model(input)

output = output.squeeze(dim=0)
estimated = torch.unbind(output, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=sample_rate))

In [ ]:
modules = {}
modules['bass'] = OpenUnmix.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='bass')
modules['drums'] = OpenUnmix.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='drums')
modules['other'] = OpenUnmix.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='other')
modules['vocals'] = OpenUnmix.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='vocals')
model = ParallelOpenUnmix(modules)
wrapper_model = ParallelOpenUnmix.TimeDomainWrapper(model, fft_size=4096, hop_size=1024, window_fn='hann')

input = mixture.unsqueeze(dim=0).unsqueeze(dim=1)

wrapper_model.eval()
with torch.no_grad():
    output = wrapper_model(input)

output = output.squeeze(dim=0)
estimated = torch.unbind(output, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=sample_rate))

In [ ]:
model = CrossNetOpenUnmix.build_from_pretrained(task="musdb18", sample_rate=sample_rate)
wrapper_model = CrossNetOpenUnmix.TimeDomainWrapper(model, fft_size=4096, hop_size=1024, window_fn='hann')

input = mixture.unsqueeze(dim=0).unsqueeze(dim=1)

wrapper_model.eval()
with torch.no_grad():
    output = wrapper_model(input)

output = output.squeeze(dim=0)
estimated = torch.unbind(output, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=sample_rate))

In [ ]:
modules = {}
modules['bass'] = D3Net.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='bass')
modules['drums'] = D3Net.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='drums')
modules['other'] = D3Net.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='other')
modules['vocals'] = D3Net.build_from_pretrained(task="musdb18", sample_rate=sample_rate, target='vocals')
model = ParallelD3Net(modules)
wrapper_model = ParallelD3Net.TimeDomainWrapper(model, fft_size=4096, hop_size=1024, window_fn='hann')

input = mixture.unsqueeze(dim=0).unsqueeze(dim=1)

wrapper_model.eval()
with torch.no_grad():
    output = wrapper_model(input)

output = output.squeeze(dim=0)
estimated = torch.unbind(output, dim=0)

for idx in range(n_sources):
    display(ipd.Audio(estimated[idx], rate=sample_rate))